# 1. Do some machine learning

## 1.0. The data we use: Palmer Pinguins

Data were collected and made available by Dr. Kristen Gorman and the Palmer Station, Antarctica LTER, a member of the Long Term Ecological Research Network. It provides a great dataset for data exploration & visualization, as an alternative to iris.

We will use this dataset in classification setting to predict the penguins’ species from anatomical information.

Each penguin is from one of the three following species: Adelie, Gentoo, and Chinstrap.

![palmer penguins](../resources/palmer_penguins.png "Palmer Penguins")


This problem is a classification problem since the target is categorical. We will use features based on penguins’ culmen measurement.

![Culmen features](../resources/culmen_depth.png)

## 1.1. Load and prepare the data

In [1]:
import pandas as pd

culmen_columns = ["Culmen Length (mm)", "Culmen Depth (mm)"]
target_column = "Species"

data_path = "../data/penguins_classification.csv"
data = pd.read_csv(data_path)

data.sample(5)

,Culmen Length (mm),Culmen Depth (mm),Species
290,50.3,20.0,Chinstrap
75,40.9,16.8,Adelie
147,36.0,17.8,Adelie
197,45.5,13.9,Gentoo
276,51.3,19.2,Chinstrap


In [2]:
from sklearn.model_selection import train_test_split

data, target = data[culmen_columns], data[target_column]
data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=0)

## 1.2. The modelling: Decision Tree

In [3]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=3, max_leaf_nodes=4)
tree.fit(data_train, target_train)

DecisionTreeClassifier(max_depth=3, max_leaf_nodes=4)

In [4]:
test_score = tree.score(data_test, target_test)
print(f"Accuracy of the DecisionTreeClassifier: {test_score:.1%}")

Accuracy of the DecisionTreeClassifier: 96.5%


# 2. Tracking experiments with MLflow

### Tracking stores

MLflow supports two types of backend stores: file store and database-backed store.

- Local file path (specified as file:/my/local/dir), where data is just directly stored locally. Defaults to `mlruns/`.
- Database encoded as +://:<password>@:/. Mlflow supports the dialects mysql, mssql, sqlite, and postgresql. For more details, see SQLAlchemy database uri.
- HTTP server (specified as https://my-server:5000), which is a server hosting an MLFlow tracking server.
- Databricks workspace (specified as databricks or as databricks://, a Databricks CLI profile.

### Artifact stores
Where you store models, plots and other stuff.

- Amazon S3
- Azure Blob Storage
- Google Cloud Storage
- FTP server
- SFTP Server
- NFS
- HDFS

Start the MLflow tracking server by running

```
mlflow server \
    --backend-store-uri /mnt/persistent-disk \
    --default-artifact-root s3://my-mlflow-bucket/ \
    --host 0.0.0.0
    --port 5000
```

or use the default storage method to write to `mlruns/`.

## 2.1. Setup and configure the tracking server

Run

```mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0 --port 5000```

in a terminal. Make sure you run it inside the virtual environment! Put `pipenv run` in front of it.

Now you should be able to see the Tracking UI in a browser at `http://0.0.0.0:5000`.

In [5]:
import mlflow
import mlflow.sklearn

In [6]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0 --port 5000
remote_server_uri = "http://0.0.0.0:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

In [7]:
mlflow.tracking.get_tracking_uri()

'http://0.0.0.0:5000'

### Create a new experiment

In [8]:
exp_name = "penguin_classification"
mlflow.create_experiment(exp_name)

'1'

## 2.2. Add tracking code to the machine learning experiment above

Basic things to track:
- Parameters: Key-value input parameters: `mlflow.log_param, mlflow.log_params`
- Metrics: Key-value metrics, where the value is numeric (can be updated over the run): `mlflow.log_metric, mlflow.log_metrics`

Recall the following code from above:

```python
# Load dataset
culmen_columns = ["Culmen Length (mm)", "Culmen Depth (mm)"]
target_column = "Species"

data_path = "../data/penguins_classification.csv"
data = pd.read_csv(data_path)

# Prepare a train-test-split
data, target = data[culmen_columns], data[target_column]
data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=0)

# Initialize and fit a classifier
tree = DecisionTreeClassifier(max_depth=3, max_leaf_nodes=4)
tree.fit(data_train, target_train)

# Calculate test scores
test_score = tree.score(data_test, target_test)
print(f"Accuracy of the DecisionTreeClassifier: {test_score:.1%}")
```


Now we add the required code to track the experiment with MLflow.

In [9]:
mlflow.set_experiment(exp_name)  # <-- set the experiment we want to track to
with mlflow.start_run() as run:         # <-- start a run of the experiment
    print(f"Started run {run.info.run_id}")
    # Load dataset
    print("Load dataset...")
    culmen_columns = ["Culmen Length (mm)", "Culmen Depth (mm)"]
    target_column = "Species"

    data_path = "../data/penguins_classification.csv"
    data = pd.read_csv(data_path)

    # Prepare a train-test-split
    print("Prepare a train-test-split...")
    data, target = data[culmen_columns], data[target_column]
    data_train, data_test, target_train, target_test = train_test_split(
        data, target, random_state=0)

    # Initialize and fit a classifier
    max_depth = 3
    max_leaf_nodes = 4
    print(f"Initialize and fit a DecisionTreeClassifier with max_depth={max_depth}, max_leaf_nodes{max_leaf_nodes}")
    
    mlflow.log_params(            # <-- Track parameters
        {"max_depth": max_depth, 
         "max_leaf_nodes": max_leaf_nodes}
    )
    tree = DecisionTreeClassifier(
        max_depth=max_depth,
        max_leaf_nodes=max_leaf_nodes
    )
    tree.fit(data_train, target_train)

    # Calculate test scores
    test_score = tree.score(data_test, target_test)
    mlflow.log_metric("test_accuracy", test_score)   # <-- Track metrics
    print(f"Result: Accuracy of the DecisionTreeClassifier: {test_score:.1%}")

Started run d26c2e3a295344b5b97bbf174994fb7c
Load dataset...
Prepare a train-test-split...
Initialize and fit a DecisionTreeClassifier with max_depth=3, max_leaf_nodes4
Result: Accuracy of the DecisionTreeClassifier: 96.5%


Have a look at the tracking UI to see how it played out!

## 2.3. Exercise: track some more stuff

What else could we want to track?

Examples:  
- Code Version: Git commit hash used for the run (if it was run from an MLflow Project)
- Start & End Time: Start and end time of the run
- Source: what code run?
- Plots.
- Properties of the input data
- Model artifacts
- ...

Track the properties of the input data as parameters. (Hint: `data.shape[0]` gives the number of samples in a dataframe)

In [10]:
# SOLUTION:

mlflow.set_experiment(exp_name)  # <-- set the experiment we want to track to
with mlflow.start_run() as run:         # <-- start a run of the experiment
    print(f"Started run {run.info.run_id}")
    # Load dataset
    print("Load dataset...")
    culmen_columns = ["Culmen Length (mm)", "Culmen Depth (mm)"]
    target_column = "Species"

    data_path = "../data/penguins_classification.csv"
    data = pd.read_csv(data_path)
    mlflow.log_param("num_samples", data.shape[0])  # <-- ADDED: track the number of samples in the dataset

    # Prepare a train-test-split
    print("Prepare a train-test-split...")
    data, target = data[culmen_columns], data[target_column]
    data_train, data_test, target_train, target_test = train_test_split(
        data, target, random_state=0)

    # Initialize and fit a classifier
    max_depth = 3
    max_leaf_nodes = 4
    print(f"Initialize and fit a DecisionTreeClassifier with max_depth={max_depth}, max_leaf_nodes{max_leaf_nodes}")
    
    mlflow.log_params(            # <-- Track parameters
        {"max_depth": max_depth, 
         "max_leaf_nodes": max_leaf_nodes}
    )
    tree = DecisionTreeClassifier(
        max_depth=max_depth,
        max_leaf_nodes=max_leaf_nodes
    )
    tree.fit(data_train, target_train)

    # Calculate test scores
    test_score = tree.score(data_test, target_test)
    mlflow.log_metric("test_accuracy", test_score)   # <-- Track metrics
    print(f"Result: Accuracy of the DecisionTreeClassifier: {test_score:.1%}")

Started run ba843b7a1a804ee1b4c325ea61cc66fe
Load dataset...
Prepare a train-test-split...
Initialize and fit a DecisionTreeClassifier with max_depth=3, max_leaf_nodes4
Result: Accuracy of the DecisionTreeClassifier: 96.5%


## 2.4. Log the model

We want to store the model artifacts to reuse it for deployment or later experimentation.
Since we used a scikit-learn model here, we can use the build-in module to store the model in sklearn format. 

```mlflow.sklearn.log_model(tree, "model")```

There are buildin modules for all kind of types of models, as well as the possibility to specify a custom format. Even autologging is available!

In [11]:
mlflow.set_experiment(exp_name)  # <-- set the experiment we want to track to
with mlflow.start_run() as run:         # <-- start a run of the experiment
    print(f"Started run {run.info.run_id}")
    # Load dataset
    print("Load dataset...")
    culmen_columns = ["Culmen Length (mm)", "Culmen Depth (mm)"]
    target_column = "Species"

    data_path = "../data/penguins_classification.csv"
    data = pd.read_csv(data_path)
    mlflow.log_param("num_samples", data.shape[0])  # <-- ADDED: track the number of samples in the dataset

    # Prepare a train-test-split
    print("Prepare a train-test-split...")
    data, target = data[culmen_columns], data[target_column]
    data_train, data_test, target_train, target_test = train_test_split(
        data, target, random_state=0)

    # Initialize and fit a classifier
    max_depth = 3
    max_leaf_nodes = 4
    print(f"Initialize and fit a DecisionTreeClassifier with max_depth={max_depth}, max_leaf_nodes{max_leaf_nodes}")
    
    mlflow.log_params(            # <-- Track parameters
        {"max_depth": max_depth, 
         "max_leaf_nodes": max_leaf_nodes}
    )
    tree = DecisionTreeClassifier(
        max_depth=max_depth,
        max_leaf_nodes=max_leaf_nodes
    )
    tree.fit(data_train, target_train)

    # Calculate test scores
    test_score = tree.score(data_test, target_test)
    mlflow.log_metric("test_accuracy", test_score)   # <-- Track metrics
    print(f"Result: Accuracy of the DecisionTreeClassifier: {test_score:.1%}")
    
    # Log the model
    mlflow.sklearn.log_model(tree, "model")

Started run 869bc120e8df4037bf91ffe1002815c6
Load dataset...
Prepare a train-test-split...
Initialize and fit a DecisionTreeClassifier with max_depth=3, max_leaf_nodes4
Result: Accuracy of the DecisionTreeClassifier: 96.5%


## 2.5 Use the model to predict locally

In [16]:
import mlflow
logged_model = 'runs:/869bc120e8df4037bf91ffe1002815c6/model'

# Load model as a Sklearn Model.
loaded_model = mlflow.sklearn.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(data_test))

array(['Adelie', 'Chinstrap', 'Adelie', 'Chinstrap', 'Adelie',
       'Chinstrap', 'Gentoo', 'Adelie', 'Gentoo', 'Adelie', 'Adelie',
       'Gentoo', 'Gentoo', 'Adelie', 'Adelie', 'Gentoo', 'Gentoo',
       'Gentoo', 'Chinstrap', 'Adelie', 'Chinstrap', 'Adelie',
       'Chinstrap', 'Gentoo', 'Adelie', 'Adelie', 'Gentoo', 'Chinstrap',
       'Gentoo', 'Gentoo', 'Adelie', 'Adelie', 'Adelie', 'Gentoo',
       'Gentoo', 'Adelie', 'Chinstrap', 'Gentoo', 'Chinstrap', 'Adelie',
       'Adelie', 'Adelie', 'Adelie', 'Gentoo', 'Adelie', 'Adelie',
       'Chinstrap', 'Gentoo', 'Gentoo', 'Chinstrap', 'Adelie', 'Adelie',
       'Adelie', 'Adelie', 'Adelie', 'Adelie', 'Gentoo', 'Adelie',
       'Adelie', 'Chinstrap', 'Adelie', 'Adelie', 'Adelie', 'Gentoo',
       'Adelie', 'Adelie', 'Adelie', 'Gentoo', 'Adelie', 'Gentoo',
       'Adelie', 'Chinstrap', 'Adelie', 'Adelie', 'Gentoo', 'Adelie',
       'Chinstrap', 'Adelie', 'Adelie', 'Gentoo', 'Adelie', 'Gentoo',
       'Chinstrap', 'Adelie', 'Chinstrap